# Sizing

Size a solar heating system - let the optimizer decide equipment sizes.

This notebook introduces:

- **InvestParameters**: Define investment decisions with size bounds and costs
- **Investment costs**: Fixed costs and size-dependent costs
- **Optimal sizing**: Let the optimizer find the best equipment sizes
- **Trade-off analysis**: Balance investment vs. operating costs

## Setup

In [1]:
import pandas as pd
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## System Description

The swimming pool heating system:

- **Solar collectors**: Convert solar radiation to heat (size to be optimized)
- **Gas boiler**: Backup heating when solar is insufficient (existing, 200 kW)
- **Buffer tank**: Store excess solar heat (size to be optimized)
- **Pool**: Constant heat demand of 150 kW during operating hours

```
   ☀️ Solar ──► [Heat] ◄── Boiler ◄── [Gas]
                  │
                  ▼
              Buffer Tank
                  │
                  ▼
                Pool 🏊
```

## Define Time Horizon and Profiles

We model one representative summer week:

In [2]:
from data.tutorial_data import get_investment_data

data = get_investment_data()
timesteps = data['timesteps']
solar_profile = data['solar_profile']
pool_demand = data['pool_demand']
GAS_PRICE = data['gas_price']
SOLAR_COST_WEEKLY = data['solar_cost_per_kw_week']
TANK_COST_WEEKLY = data['tank_cost_per_kwh_week']

In [3]:
# Visualize profiles with plotly
profiles = xr.Dataset(
    {
        'Solar Profile [kW/kW]': xr.DataArray(solar_profile, dims=['time'], coords={'time': timesteps}),
        'Pool Demand [kW]': xr.DataArray(pool_demand, dims=['time'], coords={'time': timesteps}),
    }
)
profiles.plotly.line(x='time', title='Solar and Pool Profiles', height=300)

## Build the System with Investment Options

Use `InvestParameters` to define which sizes should be optimized:

In [4]:
flow_system = fx.FlowSystem(timesteps)
flow_system.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)
flow_system.add_elements(
    # === Buses ===
    fx.Bus('Heat', carrier='heat'),
    fx.Bus('Gas', carrier='gas'),
    # === Effects ===
    fx.Effect('costs', '€', 'Total Costs', is_standard=True, is_objective=True),
    # === Gas Supply ===
    fx.Source(
        'GasGrid',
        outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=GAS_PRICE)],
    ),
    # === Gas Boiler (existing, fixed size) ===
    fx.linear_converters.Boiler(
        'GasBoiler',
        thermal_efficiency=0.92,
        thermal_flow=fx.Flow('Heat', bus='Heat', size=200),  # 200 kW existing
        fuel_flow=fx.Flow('Gas', bus='Gas'),
    ),
    # === Solar Collectors (size to be optimized) ===
    fx.Source(
        'SolarCollectors',
        outputs=[
            fx.Flow(
                'Heat',
                bus='Heat',
                # Investment optimization: find optimal size between 0-500 kW
                size=fx.InvestParameters(
                    minimum_size=0,
                    maximum_size=500,
                    effects_of_investment_per_size={'costs': SOLAR_COST_WEEKLY},
                ),
                # Solar output depends on radiation profile
                fixed_relative_profile=solar_profile,
            )
        ],
    ),
    # === Buffer Tank (size to be optimized) ===
    fx.Storage(
        'BufferTank',
        # Investment optimization: find optimal capacity between 0-2000 kWh
        capacity_in_flow_hours=fx.InvestParameters(
            minimum_size=0,
            maximum_size=2000,
            effects_of_investment_per_size={'costs': TANK_COST_WEEKLY},
        ),
        initial_charge_state=0,
        eta_charge=0.95,
        eta_discharge=0.95,
        relative_loss_per_hour=0.01,  # 1% loss per hour
        charging=fx.Flow('Charge', bus='Heat', size=200),
        discharging=fx.Flow('Discharge', bus='Heat', size=200),
    ),
    # === Pool Heat Demand ===
    fx.Sink(
        'Pool',
        inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=pool_demand)],
    ),
)

## Run Optimization

In [5]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01));

## Analyze Investment Decisions

### Optimal Sizes

In [6]:
solar_size = flow_system.statistics.sizes['SolarCollectors(Heat)'].item()
tank_size = flow_system.statistics.sizes['BufferTank'].item()

pd.DataFrame(
    {
        'Solar [kW]': solar_size,
        'Tank [kWh]': tank_size,
        'Ratio [kWh/kW]': tank_size / solar_size if solar_size > 0 else float('nan'),
    },
    index=['Optimal Size'],
).T

,Optimal Size
Solar [kW],458.451276
Tank [kWh],747.696332
Ratio [kWh/kW],1.630918


### Visualize Sizes

In [7]:
flow_system.statistics.plot.sizes()

PlotResult(data=<xarray.Dataset> Size: 16B
Dimensions:                ()
Data variables:
    BufferTank             float64 8B 747.7
    SolarCollectors(Heat)  float64 8B 458.5, figure=Figure({
    'data': [{'hovertemplate': 'variable=%{x}<br>Size=%{y}<extra></extra>',
              'legendgroup': 'BufferTank',
              'marker': {'color': '#AB63FA', 'pattern': {'shape': ''}},
              'name': 'BufferTank',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['BufferTank'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'lwGfFpJdh0A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=%{x}<br>Size=%{y}<extra></extra>',
              'legendgroup': 'SolarCollectors(Heat)',
              'marker': {'color': '#00CC96', 'pattern': {'shape': ''}},
              'name': 'SolarCollectors(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['SolarCollectors(Heat)'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'sZrBbDinfEA=', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Investment Sizes'},
               'xaxis': {'anchor': 'y',
                         'categoryarray': [BufferTank, SolarCollectors(Heat)],
                         'categoryorder': 'array',
                         'domain': [0.0, 1.0],
                         'title': {'text': 'variable'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Size'}}}
}))

### Cost Breakdown

In [8]:
total_costs = flow_system.solution['costs'].item()

# Calculate cost components
solar_invest = solar_size * SOLAR_COST_WEEKLY
tank_invest = tank_size * TANK_COST_WEEKLY
gas_costs = total_costs - solar_invest - tank_invest

pd.DataFrame(
    {
        'Solar Investment': {'EUR': solar_invest, '%': solar_invest / total_costs * 100},
        'Tank Investment': {'EUR': tank_invest, '%': tank_invest / total_costs * 100},
        'Gas Costs': {'EUR': gas_costs, '%': gas_costs / total_costs * 100},
        'Total': {'EUR': total_costs, '%': 100.0},
    }
)

,Solar Investment,Tank Investment,Gas Costs,Total
EUR,176.327414,21.568163,585.088163,782.98374
%,22.519933,2.754612,74.725455,100.00000


### System Operation

In [9]:
flow_system.statistics.plot.balance('Heat')

PlotResult(data=<xarray.Dataset> Size: 8kB
Dimensions:                (time: 169)
Coordinates:
  * time                   (time) datetime64[ns] 1kB 2024-07-15 ... 2024-07-22
Data variables:
    BufferTank(Charge)     (time) float64 1kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 nan
    BufferTank(Discharge)  (time) float64 1kB -0.0 -0.0 -0.0 ... -0.0 -0.0 nan
    GasBoiler(Heat)        (time) float64 1kB -50.0 -50.0 -50.0 ... -50.0 nan
    Pool(Heat)             (time) float64 1kB 50.0 50.0 50.0 ... 150.0 50.0 nan
    SolarCollectors(Heat)  (time) float64 1kB -0.0 -0.0 -0.0 ... -0.0 -0.0 nan, figure=Figure({
    'data': [{'fillcolor': '#AB63FA',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'BufferTank(Charge)',
              'line': {'color': '#AB63FA', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'BufferTank(Charge)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'positive',
              'type': 'scatter',
              'x': array(['2024-07-15T00:00:00.000000000', '2024-07-15T01:00:00.000000000',
                          '2024-07-15T02:00:00.000000000', '2024-07-15T03:00:00.000000000',
                          '2024-07-15T04:00:00.000000000', '2024-07-15T05:00:00.000000000',
                          '2024-07-15T06:00:00.000000000', '2024-07-15T07:00:00.000000000',
                          '2024-07-15T08:00:00.000000000', '2024-07-15T09:00:00.000000000',
                          '2024-07-15T10:00:00.000000000', '2024-07-15T11:00:00.000000000',
                          '2024-07-15T12:00:00.000000000', '2024-07-15T13:00:00.000000000',
                          '2024-07-15T14:00:00.000000000', '2024-07-15T15:00:00.000000000',
                          '2024-07-15T16:00:00.000000000', '2024-07-15T17:00:00.000000000',
                          '2024-07-15T18:00:00.000000000', '2024-07-15T19:00:00.000000000',
                          '2024-07-15T20:00:00.000000000', '2024-07-15T21:00:00.000000000',
                          '2024-07-15T22:00:00.000000000', '2024-07-15T23:00:00.000000000',
                          '2024-07-16T00:00:00.000000000', '2024-07-16T01:00:00.000000000',
                          '2024-07-16T02:00:00.000000000', '2024-07-16T03:00:00.000000000',
                          '2024-07-16T04:00:00.000000000', '2024-07-16T05:00:00.000000000',
                          '2024-07-16T06:00:00.000000000', '2024-07-16T07:00:00.000000000',
                          '2024-07-16T08:00:00.000000000', '2024-07-16T09:00:00.000000000',
                          '2024-07-16T10:00:00.000000000', '2024-07-16T11:00:00.000000000',
                          '2024-07-16T12:00:00.000000000', '2024-07-16T13:00:00.000000000',
                          '2024-07-16T14:00:00.000000000', '2024-07-16T15:00:00.000000000',
                          '2024-07-16T16:00:00.000000000', '2024-07-16T17:00:00.000000000',
                          '2024-07-16T18:00:00.000000000', '2024-07-16T19:00:00.000000000',
                          '2024-07-16T20:00:00.000000000', '2024-07-16T21:00:00.000000000',
                          '2024-07-16T22:00:00.000000000', '2024-07-16T23:00:00.000000000',
                          '2024-07-17T00:00:00.000000000', '2024-07-17T01:00:00.000000000',
                          '2024-07-17T02:00:00.000000000', '2024-07-17T03:00:00.000000000',
                          '2024-07-17T04:00:00.000000000', '2024-07-17T05:00:00.000000000',
                          '2024-07-17T06:00:00.000000000', '2024-07-17T07:00:00.000000000',
                          '2024-07-17T08:00:00.000000000', '2024-07-17T09:00:00.000000000',
                          '2024-07-17T10:00:00.000000000', '2024-07-17T11:00:00.000000000',
                          '2024-07-17T12:00:00.000000000', '2024-07-17T13:00:00.0000000

In [10]:
flow_system.statistics.plot.heatmap('SolarCollectors(Heat)')

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:    (timeframe: 8, timestep: 24)
Coordinates:
  * timeframe  (timeframe) object 64B '2024-07-15' '2024-07-16' ... '2024-07-22'
  * timestep   (timestep) object 192B '00:00' '01:00' ... '22:00' '23:00'
Data variables:
    value      (timestep, timeframe) float64 2kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 nan, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': ('timeframe: %{x}<br>timestep: %' ... '_rat<br>e: %{z}<extra></extra>'),
              'name': '0',
              'type': 'heatmap',
              'x': array(['2024-07-15', '2024-07-16', '2024-07-17', '2024-07-18', '2024-07-19',
                          '2024-07-20', '2024-07-21', '2024-07-22'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 8'}}],
    'layout': {'coloraxis': {'cmax': 350.00000000000006,
                             'cmin': 0.0,
                             'colorbar': {'title': {'text': 'SolarCollectors(Heat)|flow_rat<br>e'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

In [11]:
flow_system.statistics.plot.balance('BufferTank')

PlotResult(data=<xarray.Dataset> Size: 4kB
Dimensions:                (time: 169)
Coordinates:
  * time                   (time) datetime64[ns] 1kB 2024-07-15 ... 2024-07-22
Data variables:
    BufferTank(Charge)     (time) float64 1kB -0.0 -0.0 -0.0 ... -0.0 -0.0 nan
    BufferTank(Discharge)  (time) float64 1kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 nan, figure=Figure({
    'data': [{'fillcolor': '#e74c3c',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'BufferTank(Charge)',
              'line': {'color': '#e74c3c', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'BufferTank(Charge)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-07-15T00:00:00.000000000', '2024-07-15T01:00:00.000000000',
                          '2024-07-15T02:00:00.000000000', '2024-07-15T03:00:00.000000000',
                          '2024-07-15T04:00:00.000000000', '2024-07-15T05:00:00.000000000',
                          '2024-07-15T06:00:00.000000000', '2024-07-15T07:00:00.000000000',
                          '2024-07-15T08:00:00.000000000', '2024-07-15T09:00:00.000000000',
                          '2024-07-15T10:00:00.000000000', '2024-07-15T11:00:00.000000000',
                          '2024-07-15T12:00:00.000000000', '2024-07-15T13:00:00.000000000',
                          '2024-07-15T14:00:00.000000000', '2024-07-15T15:00:00.000000000',
                          '2024-07-15T16:00:00.000000000', '2024-07-15T17:00:00.000000000',
                          '2024-07-15T18:00:00.000000000', '2024-07-15T19:00:00.000000000',
                          '2024-07-15T20:00:00.000000000', '2024-07-15T21:00:00.000000000',
                          '2024-07-15T22:00:00.000000000', '2024-07-15T23:00:00.000000000',
                          '2024-07-16T00:00:00.000000000', '2024-07-16T01:00:00.000000000',
                          '2024-07-16T02:00:00.000000000', '2024-07-16T03:00:00.000000000',
                          '2024-07-16T04:00:00.000000000', '2024-07-16T05:00:00.000000000',
                          '2024-07-16T06:00:00.000000000', '2024-07-16T07:00:00.000000000',
                          '2024-07-16T08:00:00.000000000', '2024-07-16T09:00:00.000000000',
                          '2024-07-16T10:00:00.000000000', '2024-07-16T11:00:00.000000000',
                          '2024-07-16T12:00:00.000000000', '2024-07-16T13:00:00.000000000',
                          '2024-07-16T14:00:00.000000000', '2024-07-16T15:00:00.000000000',
                          '2024-07-16T16:00:00.000000000', '2024-07-16T17:00:00.000000000',
                          '2024-07-16T18:00:00.000000000', '2024-07-16T19:00:00.000000000',
                          '2024-07-16T20:00:00.000000000', '2024-07-16T21:00:00.000000000',
                          '2024-07-16T22:00:00.000000000', '2024-07-16T23:00:00.000000000',
                          '2024-07-17T00:00:00.000000000', '2024-07-17T01:00:00.000000000',
                          '2024-07-17T02:00:00.000000000', '2024-07-17T03:00:00.000000000',
                          '2024-07-17T04:00:00.000000000', '2024-07-17T05:00:00.000000000',
                          '2024-07-17T06:00:00.000000000', '2024-07-17T07:00:00.000000000',
                          '2024-07-17T08:00:00.000000000', '2024-07-17T09:00:00.000000000',
                          '2024-07-17T10:00:00.000000000', '2024-07-17T11:00:00.000000000',
                          '2024-07-17T12:00:00.000000000', '2024-07-17T13:00:00.000000000',
                          '2024-07-17T14:00:00.000000000', '2024-07-17T15:00:00.000000000',
                          '2024-07-17T16:00:00.000000000', '2024-07-17T17:00:00.000000000',
                          '2024-07-17T18:00:00.0

## Compare: What if No Solar?

Let's see how much the solar system saves:

In [12]:
# Gas-only scenario for comparison
total_demand = pool_demand.sum()
gas_only_cost = total_demand / 0.92 * GAS_PRICE  # All heat from gas boiler
savings = gas_only_cost - total_costs

pd.DataFrame(
    {
        'Gas-only [EUR/week]': gas_only_cost,
        'With Solar [EUR/week]': total_costs,
        'Savings [EUR/week]': savings,
        'Savings [%]': savings / gas_only_cost * 100,
        'Savings [EUR/year]': savings * 52,
    },
    index=['Value'],
).T

,Value
Gas-only [EUR/week],2465.217391
With Solar [EUR/week],782.983740
Savings [EUR/week],1682.233651
Savings [%],68.238755
Savings [EUR/year],87476.149872


### Energy Flow Sankey

A Sankey diagram visualizes the total energy flows through the system:

In [13]:
flow_system.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:  (link: 7)
Coordinates:
  * link     (link) int64 56B 0 1 2 3 4 5 6
    source   (link) <U15 420B 'Heat' 'BufferTank' ... 'Heat' 'SolarCollectors'
    target   (link) <U10 280B 'BufferTank' 'Heat' 'GasBoiler' ... 'Pool' 'Heat'
    label    (link) <U21 588B 'BufferTank(Charge)' ... 'SolarCollectors(Heat)'
    carrier  (link) <U4 112B 'heat' 'heat' 'gas' 'heat' 'gas' 'heat' 'heat'
Data variables:
    value    (link) float64 56B 5.081e+03 4.277e+03 ... 1.89e+04 1.522e+04, figure=Figure({
    'data': [{'link': {'color': [rgba(231, 76, 60, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(231, 76, 60, 0.4)],
                       'label': [BufferTank(Charge), BufferTank(Discharge),
                                 GasBoiler(Gas), GasBoiler(Heat), GasGrid(Gas),
                                 Pool(Heat), SolarCollectors(Heat)],
                       'source': [5, 3, 0, 6, 2, 5, 4],
                       'target': [3, 5, 6, 5, 0, 1, 5],
                       'value': [5081.34966709651, 4277.427193176951,
                                 4875.734690067062, 4485.675914861697,
                                 4875.734690067062, 18900.0, 15218.246559057861]},
              'node': {'color': [#3498db, #FFA15A, #636EFA, #AB63FA, #00CC96,
                                 #e74c3c, #EF553B],
                       'label': [Gas, Pool, GasGrid, BufferTank, SolarCollectors,
                                 Heat, GasBoiler],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

## Key Concepts

### InvestParameters Options

```python
fx.InvestParameters(
    minimum_size=0,           # Lower bound (can be 0 for optional)
    maximum_size=500,         # Upper bound
    fixed_size=100,           # Or: fixed size (binary decision)
    mandatory=True,           # Force investment to happen
    effects_of_investment={'costs': 1000},      # Fixed cost if invested
    effects_of_investment_per_size={'costs': 25},  # Cost per unit size
)
```

### Where to Use InvestParameters

- **Flow.size**: Optimize converter/source/sink capacity
- **Storage.capacity_in_flow_hours**: Optimize storage capacity

## Summary

You learned how to:

- Define **investment decisions** with `InvestParameters`
- Set **size bounds** (minimum/maximum)
- Add **investment costs** (per-size and fixed)
- Access **optimal sizes** via `statistics.sizes`
- Visualize sizes with `statistics.plot.sizes()`

### Next Steps

- **[04-operational-constraints](04-operational-constraints.ipynb)**: Add startup costs and minimum run times
- **[05-multi-carrier-system](05-multi-carrier-system.ipynb)**: Model combined heat and power